In [ ]:
from google.colab import files
import zipfile
from google.colab import drive

import os
import numpy as np
import cv2
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf
import matplotlib.pyplot as plt
from PIL.Image import ImageTransformHandler
from skimage import transform
from numpy.linalg import inv
from numpy import random as r
from PIL import Image
from skimage import draw
from scipy.ndimage import gaussian_filter
from google.colab.patches import cv2_imshow


In [ ]:
def detect_ball():
  params = cv2.SimpleBlobDetector_Params()
  params.minThreshold = 0
  params.maxThreshold = 256
  params.thresholdStep = 1
  params.filterByArea = True
  params.minArea = 1500
  params.filterByCircularity = False
  params.minCircularity = 0.3
  params.filterByConvexity = True
  params.minConvexity = 0.8
  params.filterByInertia = False
  # params.minInertiaRatio = 0
  # params.maxInertiaRatio = 1

  detector = cv2.SimpleBlobDetector_create(params)
  im = cv2.imread('ball_blob.jpg', cv2.IMREAD_GRAYSCALE)
  keypts = detector.detect(im)
  im_with_keypoints = cv2.drawKeypoints(im, keypts, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
  cv2_imshow(im_with_keypoints)
  return keypts, im_with_keypoints

def detect_stick():
  params = cv2.SimpleBlobDetector_Params()
  params.minThreshold = 0
  params.maxThreshold = 256
  params.thresholdStep = 1
  params.filterByArea = False
  params.minArea = 1500
  params.filterByCircularity = False
  params.minCircularity = 0.3
  params.filterByConvexity = False
  params.minConvexity = 0.8
  params.filterByInertia = False
  # params.minInertiaRatio = 0
  # params.maxInertiaRatio = 1

  detector = cv2.SimpleBlobDetector_create(params)
  im = cv2.imread('stick_blob.jpg', cv2.IMREAD_GRAYSCALE)

  keypts = detector.detect(im)
  im_with_keypoints = cv2.drawKeypoints(im, keypts, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
  cv2_imshow(im_with_keypoints)
  return keypts, im_with_keypoints

In [ ]:
stick_keys, stick_with_keypts = detect_stick()
ball_keys, ball_with_keypts = detect_ball()

ball_coords = []
for kpt in ball_keys:
  ball_coords.append(kpt.pt)

stick_pts = []
for kpt in stick_keys:
  stick_pts.append(kpt.pt)

error: ignored

In [ ]:
# CODE to get linear set of points for plotting
from statistics import mean
import numpy as np
stick_coords_x = []
stick_coords_y = []
for i in stick_pts:
  stick_coords_x.append(i[0])
  stick_coords_y.append(i[1])

stick_coords_x = np.array(stick_coords_x)
stick_coords_y = np.array(stick_coords_y)


def best_fit_line_extreme_points(x,y):
  m = (((mean(x)*mean(y)) - mean(np.multiply(x,y))) / ((mean(x)*mean(x)) - mean(np.multiply(x,x))))
  b = mean(y) - m*mean(x)
  min_x = min(x)
  max_x = max(x)
  extremepoints = []
  extremepoints.append(tuple((min_x, min_x * m + b)))
  extremepoints.append(tuple((max_x, max_x * m + b)))
  return extremepoints

best_fit_line_extreme_points(stick_coords_x, stick_coords_y)

NameError: ignored

In [ ]:
grid = np.zeros([4032, 3024])
for pt in ball_coords:
  pt_t = [pt[0], pt[1] + 100] # ball offset 100 pixels
  rr, cc = draw.disk(pt_t, 25, shape=grid.shape);
  grid[rr, cc] = 255

for pt in stick_pts:
  rr, cc = draw.disk(pt, 10, shape=grid.shape);
  grid[rr, cc] = 50

im = Image.fromarray(grid.transpose())
im.show()

NameError: ignored

In [ ]:
len(ball_coords)

12

In [ ]:
def trajectory_path(x,y):
  m = (((mean(x)*mean(y)) - mean(np.multiply(x,y))) / ((mean(x)*mean(x)) - mean(np.multiply(x,x))))
  b = mean(y) - m*mean(x)
  max_x = max(x)
  extremepoints = []
  extremepoints.append(tuple((max_x, max_x * m + b)))
  return extremepoints